In [5]:
################################################################################
#
####                               IMPORTING
#
################################################################################

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
from pathlib import Path
import pickle as pkl
import matplotlib.pyplot as plt
import scipy.io as sio
import random

import sys
sys.path.append(os.path.dirname(os.getcwd()) + '/model/oamp')
sys.path.append(os.path.dirname(os.getcwd()))
from model.oamp.oampnet import *
from model.oamp.oampnet import oampnet
from model.oamp.sample_generator import sample_generator
from model.oamp.utils_oampnet import * 
################################################################################
#
####                              SETTINGS
#
################################################################################

######################
###  General setup ###
######################
torch.manual_seed(123)
np.random.seed(123)

saveModel = False
dirPath = str(Path(os.getcwd()))
model_filename = dirPath + '/oampnet_model_3gpp16' + '.pth'

data_filename = str(Path(os.getcwd()).parent.absolute()) + '/data/H_bank.mat'
data_filename_test = str(Path(os.getcwd()).parent.absolute()) + '/data/H_bank2.mat'
useGPU = True # If true, and GPU is available, use it.

#\\\ Determine processing unit:
if useGPU and torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = 'cuda:0'
else:
    device = 'cpu'

######################
###  Load parameters of the system ###
######################
NT = 32
NR = 64
mod_n = 16

mat_contents = sio.loadmat(data_filename)
mat_contents_test = sio.loadmat(data_filename_test)

num_layers = 10
train_iter = 10000
train_batch_size = 3276
test_batch_size = 3276
mini_validtn_batch_size = 100
learning_rate = 1e-3

corr_flag = True
rho = 0.6

validtn_NT_list = np.asarray([NT])
snrdb_list = {16:np.arange(11.0, 22.0), 32:np.arange(18.0, 25.0)}

################################################################################
#
####                               MAIN RUN
#
################################################################################

######################
###  Train function ###
######################
def train(model, optimizer, generator, device='cpu'):
    H = mat_contents_test['H_bank']
    criterion = nn.MSELoss().to(device=device)
    model.train()
    real_QAM_const = generator.real_QAM_const.to(device=device)
    imag_QAM_const = generator.imag_QAM_const.to(device=device)

    Haux = torch.tensor(H[:, :, random.sample(range(100), NT)])#Pick up NT random users from 100.
    Hr = torch.real(Haux)
    Hi = torch.imag(Haux)
    h1 = torch.cat((Hr, -1. * Hi), dim=2)
    h2 = torch.cat((Hi, Hr), dim=2)
    H_test = torch.cat((h1, h2), dim=1)
    H_test = H_test.to(device=device).double()
    
    for i in range(train_iter):
        H = mat_contents['H_bank']
        Haux = torch.tensor(H[:, :, random.sample(range(100), NT)])#Pick up NT random users from 100.
        Hr = torch.real(Haux)
        Hi = torch.imag(Haux)
        h1 = torch.cat((Hr, -1. * Hi), dim=2)
        h2 = torch.cat((Hi, Hr), dim=2)
        H_train = torch.cat((h1, h2), dim=1)
        
        
        y, x, j_indices, noise_sigma = generator.give_batch_data_Hinput(H_train, NT, snr_db_min=snrdb_list[NT][0],
                                                                snr_db_max=snrdb_list[NT][-1], 
                                                                batch_size = train_batch_size)
        H_train = H_train.to(device=device).double()
        y = y.to(device=device).double()
        noise_sigma = noise_sigma.to(device=device).double()

        list_batch_x_predicted = model.forward(H_train, y, noise_sigma)

        x = x.to(device=device).double()
        j_indices = j_indices.to(device=device)

        loss, SER = loss_fn(x, list_batch_x_predicted, num_layers, j_indices, real_QAM_const, imag_QAM_const, criterion)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        del H, y, x, j_indices, noise_sigma, list_batch_x_predicted

        if (i%1000==0):
            print('iteration number : ', i, 'User : ', NT, 'loss : ', loss.item())
            print('Now validating')

            model.eval()
            y, x, j_indices, noise_sigma = generator.give_batch_data_Hinput(H_test, NT, snr_db_min=snrdb_list[NT][0],
                                                                     snr_db_max=snrdb_list[NT][0], 
                                                                     batch_size = test_batch_size)			
            y = y.to(device=device)
            j_indices = j_indices.to(device=device)
            noise_sigma = noise_sigma.to(device=device)
            with torch.no_grad():

                list_batch_x_predicted = model.forward(H_test, y, noise_sigma)
                validtn_out = list_batch_x_predicted[-1].to(device=device)
                accr = sym_detection(validtn_out, j_indices, real_QAM_const, imag_QAM_const)
                         
                
            print('Mini validation result : ', accr)

            model.train()
            torch.save(model.state_dict(), model_filename)
            print('********Model Saved******* at directory : ', model_filename)

In [7]:
######################
###  Run training ###
######################
device = 'cuda'
generator = sample_generator(train_batch_size, mod_n, NR)
model = oampnet(num_layers, generator.constellation, generator.real_QAM_const, generator.imag_QAM_const, device=device)
model = model.to(device=device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train(model, optimizer, generator, device)

iteration number :  0 User :  32 loss :  0.0191435926322349
Now validating
Mini validation result :  0.8127003205128205
********Model Saved******* at directory :  /home/nicolas/MIMO_detection_project/Langevin_repo/Langevin-MIMO-detector/notebooks/oampnet_model_3gpp16.pth


KeyboardInterrupt: 

In [8]:
from collections import defaultdict
import torch
import torch.nn as nn
import torch.nn.functional as Fpero
import numpy as np
import os
from pathlib import Path
import pickle as pkl
import matplotlib.pyplot as plt
import scipy.io as sio
import random

import sys
sys.path.append(os.path.dirname(os.getcwd()) + '/model/oamp')
sys.path.append(os.path.dirname(os.getcwd()))
from model.oamp.oampnet import *
from model.oamp.oampnet import oampnet
from model.oamp.sample_generator import sample_generator
from model.oamp.utils_oampnet import * 

################################################################################
#
####                              SETTINGS
#
################################################################################

######################
###  General setup ###
######################
torch.manual_seed(123)
np.random.seed(123)

useGPU = True # If true, and GPU is available, use it.
#\\\ Determine processing unit:
if useGPU and torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = 'cuda:0'
else:
    device = 'cpu'

dirPath = str(Path(os.getcwd()))
model_filename = dirPath + '/oampnet_model_3gpp16' + '.pth'
dirMainPath = str(Path(os.getcwd()).parent.absolute())

data_filename = str(Path(os.getcwd()).parent.absolute()) + '/data/H_bank.mat'
data_filename_test = str(Path(os.getcwd()).parent.absolute()) + '/data/H_bank2.mat'

NT = 32
NR = 64
mod_n = 16
num_layers = 10
corr_flag = True
rho = 0.6

validtn_NT_list = np.asarray([NT])
snrdb_list = {16:np.arange(11.0, 22.0), 32:np.arange(18.0, 25.0)}


################################################################################
#
####                              DATA
#
################################################################################

mat_contents = sio.loadmat(data_filename)
mat_contents_test = sio.loadmat(data_filename_test)

channelInput = True
H = mat_contents_test['H_bank']
Haux = torch.tensor(H[:, :, random.sample(range(100), NT)])#Pick up NT random users from 100.
Hr = torch.real(Haux)
Hi = torch.imag(Haux)
h1 = torch.cat((Hr, -1. * Hi), dim=2)
h2 = torch.cat((Hi, Hr), dim=2)
H_test = torch.cat((h1, h2), dim=1)
H_test = H_test.to(device=device).double()

test_batch_size = 3276

################################################################################
#
####                              MAIN RUN
#
################################################################################

def test(model, generator, device):
    # Testing Trained Network
    accs_NN = model_eval(NT, model, snrdb_list[NT][-2], snrdb_list[NT][-1], test_batch_size,  generator, device, num_layers, channel_input = channelInput, H_test_set = H_test)
    return accs_NN

generator = sample_generator(test_batch_size, mod_n, NR)
model = oampnet(num_layers, generator.constellation, generator.real_QAM_const, generator.imag_QAM_const, device=device)
model = model.to(device=device)
model.load_state_dict(torch.load(model_filename))
print('*******Successfully loaded pre-trained model***********')

results = test(model, generator, device)

print(results)

serlist = []
for ii in range(len(snrdb_list[NT])):
    serlist.append(results[ii][1])

with open(dirMainPath + '/results/oampnet_results', "wb") as output_file:
    pkl.dump(serlist, output_file)

*******Successfully loaded pre-trained model***********
[(23.0, 0.001317918192918377), (24.0, 0.0005427731990230722)]


IndexError: list index out of range